In [1]:
import os
print(os.getcwd())

models_dir = 'models'
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    print(f"{models_dir} klasörü oluşturuldu.")
else:
    print(f"{models_dir} klasörü zaten var.")

!pip install torchviz graphviz


/content
models klasörü oluşturuldu.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Suc

In [28]:
import os
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import shutil
from google.colab import files
from IPython.display import display, HTML, Javascript

# "/content/models/" dizinindeki tüm dosya adlarını listele
model_directory = '/content/models/'

# Dosya adı listesi oluştur
model_files = [f for f in os.listdir(model_directory) if os.path.isfile(os.path.join(model_directory, f))]

# Dosya varsa listeyi yazdır, yoksa uyarı ver
if model_files:
    print("Mevcut model dosyaları:", model_files)
else:
    print("Uyarı: 'models' dizininde hiçbir dosya bulunamadı.")

# Modeli tanımla (DQN)
class DQN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DQN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        return self.network(x)

# Ağırlıkları ve ağı test et
weights = []
for idx, model_file in enumerate(model_files):
    print(f"Yükleniyor: {model_file}")
    checkpoint = torch.load(os.path.join(model_directory, model_file))

    model = DQN(input_size=21, hidden_size=256, output_size=3)
    model.load_state_dict(checkpoint["model_state_dict"], strict=False)

    x = torch.randn(1, 21)
    y = model(x)

    initial_weights = model.network[0].weight.detach().numpy()
    weights.append(initial_weights)

    if len(model_files) > 1 and (idx + 1) % (max(1, len(model_files) // 10)) == 0:
        plt.figure(figsize=(10, 6))
        plt.plot(weights[idx][0], label=f"Iterasyon {idx + 1}")
        plt.xlabel("Ağırlık Endeksi")
        plt.ylabel("Ağırlık Değeri")
        plt.title(f"Ağırlıkların Değişimi: {model_file}")
        plt.legend()
        plt.savefig(f"/content/{model_file}_weights_change.png")
        plt.close()

print("Grafikler oluşturuldu.")

import zipfile
# ZIP dosyası için hedef yol
zip_path = "/content/weights_plots.zip"

# ZIP dosyasını oluştur ve yalnızca oluşturulan grafikleri ekle
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file in os.listdir("/content/"):
        if file.endswith("_weights_change.png"):  # Sadece oluşturulan grafikler
            zipf.write(os.path.join("/content/", file), arcname=file)

# ZIP dosyasını indir
files.download(zip_path)


Mevcut model dosyaları: ['model_checkpoint_1000.pth', 'model_checkpoint_17900.pth', 'model_checkpoint_17500.pth']
Yükleniyor: model_checkpoint_1000.pth


<ipython-input-28-f7757640d0ba>:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(os.path.join(model_directory, model_file))


Yükleniyor: model_checkpoint_17900.pth
Yükleniyor: model_checkpoint_17500.pth
Grafikler oluşturuldu.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>